In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
from pandas import plotting
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly import tools
init_notebook_mode(connected = True)
import plotly.figure_factory as ff

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#reading de dataset
data= pd.read_csv(r'/kaggle/input/customer-segmentation-tutorial-in-python/Mall_Customers.csv', delimiter = ",")

In [ ]:
data.head()

In [ ]:
#statistical Analysis
data.describe()

In [ ]:
data.dtypes

In [ ]:
#histogram

data['Age'].plot.hist()
plt.show()

In [ ]:
data['Annual Income (k$)'].plot.hist()
plt.show()

In [ ]:
#dataset variable distribuition

box = data.iloc[:, [2,3,4]].plot.box()
box.set_title('Distribucion Variables')
plt.show()

#there is an atipic value in our annual income variable

In [ ]:
#feature engienering , new column base on the variable age 

data['decade'] = (data['Age']//10)*10

In [ ]:
data.head()

In [ ]:
#gender Histogram by decade (age)

pd.crosstab(index = data['Gender'], columns = data["decade"]).plot(kind='bar')
plt.show()


In [ ]:
#correlation heatmap analysis

f,ax = plt.subplots(figsize=(9,9))
sns.heatmap(data.corr(method='spearman'),annot=True,vmin=-1, vmax=1, center= 0)
plt.show()

In [ ]:
#scatter analysis 

data.plot(kind = "scatter", x = "Annual Income (k$)", y = "Spending Score (1-100)",)
plt.show()

#there is no correlation between this two variables therefore we can use them in our clustering algorithm

In [ ]:
#scatter analysis 

data.plot(kind = "scatter", x = "Spending Score (1-100)", y = "Age",)
plt.show()

#there is no correlation between this two variables therefore we can use them in our clustering algorithm, with these variables ill use hierarchy clusterization

In [ ]:
#slicing the data

dataset = data.iloc[:,[4,3]].values

#Spending Score
#annual Income

In [ ]:
X= dataset 
print(X)

In [ ]:
#Kmeans , elbow method

from sklearn.cluster import KMeans

from sklearn.cluster import KMeans

def elbow_method(epsilon, figure= False):
    wcss = []


    diff = np.inf
    i = 1

    while diff > epsilon:
      print("iteracion No Clusters: k: {k}".format(k=i))



      kmeans = KMeans(n_clusters = i, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
      kmeans.fit(X)


      if diff== np.inf:
          diff = kmeans.inertia_

      elif kmeans.inertia_== 0:
        wcss.append(kmeans.inertia_)
        break

      else:
        diff = (wcss [-1] - kmeans.inertia_)/wcss[-1]
      wcss.append(kmeans.inertia_)
      i += 1

    if figure:
      plt.plot(range(0,len(wcss)),wcss)
      plt.title('elbow_method')
      plt.xlabel('Clusters')
      plt.ylabel('WCSS')


    k = i-1
    return wcss,k

In [ ]:
#Cluster identification

epsilon= 0.05
wcss, _=elbow_method(epsilon, figure= True)

In [ ]:
epsilon = 0.33
_, k = elbow_method(epsilon, figure= False)

#Cluster value recommendation based on epsilon value 

In [ ]:
Kmeans = KMeans(n_clusters = k, init = 'k-means++', max_iter = 300, n_init =  10, random_state = 0)
y_Kmeans = Kmeans.fit_predict(X)

In [ ]:
plt.scatter(X[y_Kmeans == 0, 0], X[y_Kmeans == 0, 1], s = 100, c = 'blue', label = 'C0')
plt.scatter(X[y_Kmeans == 1, 0], X[y_Kmeans == 1, 1], s = 100, c = 'magenta', label = 'C1')


plt.scatter(Kmeans.cluster_centers_[:, 0], Kmeans.cluster_centers_[:,1], s = 300, c = 'black', label = 'Centroids')
plt.title('Kmeans Clusters')
plt.xlabel('Spending Score')
plt.ylabel('Annual Income')
plt.legend()
plt.show()

In [ ]:
#hierarchy clusterization with X values 

import scipy.cluster.hierarchy as sch

In [ ]:
dendrogram = sch.dendrogram(sch.linkage(X, method = 'ward'))
plt.title('Dendrogam', fontsize = 20)
plt.xlabel('Mall_Customers')
plt.ylabel('Ecuclidean Distance')
plt.show()

In [ ]:
from sklearn.cluster import AgglomerativeClustering
hc = AgglomerativeClustering(n_clusters = 5, affinity = 'euclidean', linkage = 'ward')
y_hc = hc.fit_predict(X)


In [ ]:
plt.scatter(X[y_hc == 0, 0], X[y_hc == 0, 1], s = 100, c = 'red', label = 'C0')
plt.scatter(X[y_hc == 1, 0], X[y_hc == 1, 1], s = 100, c = 'yellow', label = 'C1')
plt.scatter(X[y_hc == 2, 0], X[y_hc == 2, 1], s = 100, c = 'cyan', label = 'C2')
plt.scatter(X[y_hc == 3, 0], X[y_hc == 3, 1], s = 100, c = 'magenta', label = 'C3')
plt.scatter(X[y_hc == 4, 0], X[y_hc == 4, 1], s = 100, c = 'blue', label = 'C4')

plt.style.use('fivethirtyeight')
plt.title('Hierarchial Clustering', fontsize = 20)
plt.xlabel('Spending Score')
plt.ylabel('Annual Income')
plt.legend()
plt.grid()
plt.show()

In [ ]:
#bringing our results into the data

data['Kmeans'] = y_Kmeans

In [ ]:
data['Hierarchy'] = y_hc

In [ ]:
data.head()

#we can observe the cluster number tht was assigng to every individual observation , this allows us to analyze the caracteristics of every cluster

In [ ]:
data.groupby('Kmeans').size().plot(kind = "bar")
plt.title('Kmeans Simulation')
plt.show()

In [ ]:
data.groupby('Hierarchy').size().plot(kind = "bar")
plt.title('Hierarchy Simulation')
plt.show()

In [ ]:
Cluster0 = (data[data['Kmeans']== 0])

In [ ]:
Cluster0.iloc[:,[0,1,2,3,4,5]].describe()

#we can obtain individual statistical values from every cluster and compare them

In [ ]:
Cluster1 = (data[data['Kmeans']== 1])

In [ ]:
Cluster1.iloc[:,[0,1,2,3,4,5]].describe()